In [1]:
cd ..

c:\Users\npram\Desktop\EPITA\CLASS and NOTES\SEM2\Data_science_production\dsp-correction


In [1]:

import numpy as np
import pandas as pd
import mlflow

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

In [4]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [5]:
mlflow.set_experiment("House price prediction data mlflow")

INFO: 'House price prediction data mlflow' does not exist. Creating a new experiment


In [6]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

def get_dataset() -> pd.DataFrame:
    db = load_diabetes()
    X, y = db.data, db.target
    return train_test_split(X, y, random_state=42)

X_train, X_test, y_train, y_test = get_dataset()

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #mlflow.log_metric('rmse', rmse)
    #mlflow.log_metric('mae', mae)
    #mlflow.log_metric('r2', r2)
    mlflow.log_metrics({'rmse': rmse, 'mae': mae, 'r2': r2})
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R2 = {r2:.2f}')
    return rmse, mae, r2

def train_model(X_train, X_test, y_train, y_test: pd.DataFrame, model_class, **model_kwargs) -> int:
    model = model_class(**model_kwargs)
    mlflow.log_params(model_kwargs)
    model.fit(X_train, y_train)
    mlflow.sklearn.log_model(model, 'elastic_net')
    evaluate_model(model, X_test, y_test)

In [7]:

with mlflow.start_run():
    model_kwargs = {'alpha': 0.01, 'l1_ratio': 0.75}
    train_model(X_train, X_test, y_train, y_test, ElasticNet, **model_kwargs)

RMSE = 55.11, MAE = 45.22, R2 = 0.45
